# Create LUVOIR B aperture

Create a LUVOIR B aperture file, also with indexed aperture and segment positions in the file header.

In [ ]:
from astropy.io import fits
import hcipy
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Create a pupil grid on which we will sample the aperture
pup_grid = hcipy.make_pupil_grid((1024, 1024), 8.188)

## Create segmented aperture and header

In [ ]:
# Create the aperture generater, and evaluate it in a supersampled pupil grid to get the actual array
# Also create the header that contains some basic info about the aperture
luvoir_b_generator, header = hcipy.aperture.realistic.make_luvoir_b_aperture(return_header=True)
luvoir_b = hcipy.evaluate_supersampled(luvoir_b_generator, pup_grid, 10)

In [ ]:
# Plot the aperture
plt.figure(figsize=(8,8))
hcipy.imshow_field(luvoir_b, cmap='Greys_r')

In [ ]:
# Write aperture to file
hcipy.write_fits(luvoir_b, 'aperture_LUVOIR-B.fits')

## Create indeced aperture and get segment positions

In [ ]:
# Create indices for 55 segments, starting with 0 on center segment
indices = np.arange(55)

#### ATTENTION !

It's at this point that you have to hack the `hcipy` function `make_luvoir_b_aperture()` to return the list `segment_positions` which contains the segment center coordinates, rather than the segment generators `segments`.

In [ ]:
# Create the indexed aperture generater, and evaluate it on a supersampled pupil grid to get the actual array
luvoir_b_generator_indexed, segs = hcipy.aperture.realistic.make_luvoir_b_aperture(segment_transmissions=indices,
                                                                                   return_segments=True)
luvoir_b_indexed = hcipy.evaluate_supersampled(luvoir_b_generator_indexed, pup_grid, 10)

In [ ]:
# Plot the indexed segmented aperture and test x/y seg coordiantes with first three segments
plt.figure(figsize=(8,8))
hcipy.imshow_field(luvoir_b_indexed)
for i, pair in zip(range(3), segs):
    plt.plot(pair[0], pair[1], marker='o')

In [ ]:
# Check what format the header is - it's a dictionary
header

In [ ]:
# Make an actual fits header out of it
header_full = fits.Header(header.items())

In [ ]:
# Add all segment positions to header
for i, pair in zip(range(55), segs):
    #print(f'Index: {i}')
    #print(f'X: {pair[0]}')
    #print(f'Y: {pair[1]}')
    header_full.set(f'SEG{i}_X', pair[0], f'x-position segment {i}')
    header_full.set(f'SEG{i}_Y', pair[1], f'y-position segment {i}') 

In [ ]:
# Write indexed aperture to file, with header containing segment coordinates
fits.writeto('aperture_LUVOIR-B_indexed.fits', luvoir_b_indexed.shaped, header_full, overwrite=True)